In [ ]:
import pandas as pd

# List of filenames 
file_names = ['df_Customers.csv', 'df_OrderItems.csv', 'df_Orders.csv', 'df_Payments.csv', 'df_Products.csv']

# Loop through each file and merge
for file in file_names:
    train_path = f'data/train/{file}'
    test_path = f'data/test/{file}'

    # Read both CSV files
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # Merge them
    merged_df = pd.concat([train_df, test_df], ignore_index=True)

    # Save to a new merged file
    new_file_name = file.replace('df_', '')
    merged_df.to_csv(f'{new_file_name}', index=False)
    print(f'Merged: {file} → {new_file_name}')

         customer_id  customer_zip_code_prefix        customer_city  \
0       hCT0x9JiGXBQ                     58125      varzea paulista   
1       PxA7fv9spyhx                      3112   armacao dos buzios   
2       g3nXeJkGI0Qw                      4119              jandira   
3       EOEsCQ6QlpIg                     18212           uberlandia   
4       mVz5LO2Vd6cL                     88868             ilhabela   
...              ...                       ...                  ...   
127590  Dw3aCTFf4Q4G                     26160            guarulhos   
127591  JmHJqJvpVcJs                     39628       rio de janeiro   
127592  ro5DikwWCC3j                     89284  santana de parnaiba   
127593  R25nE2rl77AU                     60867            sao paulo   
127594  oOvkrmF7ZZbI                     37740              diadema   

       customer_state  
0                  SP  
1                  RJ  
2                  SP  
3                  MG  
4                  SP  
...

## Data Cleaning

### Customer Data

In [3]:
# import customers data 
customers = pd.read_csv('data/customers.csv')
customers.head(5) 

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,hCT0x9JiGXBQ,58125,varzea paulista,SP
1,PxA7fv9spyhx,3112,armacao dos buzios,RJ
2,g3nXeJkGI0Qw,4119,jandira,SP
3,EOEsCQ6QlpIg,18212,uberlandia,MG
4,mVz5LO2Vd6cL,88868,ilhabela,SP


In [20]:
customers.info() # check the data types and null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127595 entries, 0 to 127594
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   customer_id               127595 non-null  object
 1   customer_zip_code_prefix  127595 non-null  int64 
 2   customer_city             127595 non-null  object
 3   customer_state            127595 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.9+ MB


In [11]:
# check for duplicates
duplicates = customers.duplicated().sum()
duplicates

0

### Orders Data

In [ ]:
# import orders data
orders = pd.read_csv('data/orders.csv')
orders.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18


In [ ]:
orders.info() # check for data types and null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127595 entries, 0 to 127594
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   order_id                       127595 non-null  object
 1   customer_id                    127595 non-null  object
 2   order_status                   89316 non-null   object
 3   order_purchase_timestamp       127595 non-null  object
 4   order_approved_at              127579 non-null  object
 5   order_delivered_timestamp      87427 non-null   object
 6   order_estimated_delivery_date  89316 non-null   object
dtypes: object(7)
memory usage: 6.8+ MB


In [13]:
orders.isnull().sum() # check for missing values

order_id                             0
customer_id                          0
order_status                     38279
order_purchase_timestamp             0
order_approved_at                   16
order_delivered_timestamp        40168
order_estimated_delivery_date    38279
dtype: int64

In [15]:
# check for unique values in order_status
orders['order_status'].value_counts() 

delivered      87428
shipped          936
canceled         409
processing       273
invoiced         266
unavailable        2
approved           2
Name: order_status, dtype: int64

Here, we replace null values of `order_status` with "unavailable" as we don't have sufficient information.

In [16]:
# impute the null values with 'unavailable'
orders['order_status'].fillna('unavailable', inplace=True)
orders.isnull().sum() # check for missing values

order_id                             0
customer_id                          0
order_status                         0
order_purchase_timestamp             0
order_approved_at                   16
order_delivered_timestamp        40168
order_estimated_delivery_date    38279
dtype: int64

Here, we will not replace the null values in others column as it is justifiable that order's approved time, delivered time and estimated delivery date can be empty is order is just placed or in process.

In [21]:
# check for duplicates
duplicates = orders.duplicated().sum()
duplicates

0

In [ ]:
# save the cleaned orders data
orders.to_csv('data/orders.csv', index=False)

### Order Items Data

In [ ]:
# import orderitems data
orderitems = pd.read_csv('data/orderitems.csv')
orderitems.head(5)

,order_id,product_id,seller_id,price,shipping_charges
0,Axfy13Hk4PIk,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65
1,v6px92oS8cLG,qejhpMGGVcsl,IjlpYfhUbRQs,170.80,23.79
2,Ulpf9skrhjfm,qUS5d2pEAyxJ,77p2EYxcM9MD,64.40,17.38
3,bwJVWupf2keN,639iGvMyv0De,jWzS0ayv9TGf,264.50,30.72
4,Dd0QnrMk9Cj5,1lycYGcsic2F,l1pYW6GBnPMr,779.90,30.66


In [19]:
orderitems.info() # check for data types and null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127595 entries, 0 to 127594
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          127595 non-null  object 
 1   product_id        127595 non-null  object 
 2   seller_id         127595 non-null  object 
 3   price             127595 non-null  float64
 4   shipping_charges  127595 non-null  float64
dtypes: float64(2), object(3)
memory usage: 4.9+ MB


In [24]:
# check for duplicates
duplicates = orderitems.duplicated().sum()
duplicates

0         False
1         False
2         False
3         False
4         False
          ...  
127590    False
127591    False
127592    False
127593    False
127594    False
Length: 127595, dtype: bool

### Products Data

In [ ]:
# import products data 
products = pd.read_csv('data/products.csv')
products.head(5) 

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,90K0C1fIyQUf,toys,491.0,19.0,12.0,16.0
1,qejhpMGGVcsl,watches_gifts,440.0,18.0,14.0,17.0
2,qUS5d2pEAyxJ,costruction_tools_garden,2200.0,16.0,16.0,16.0
3,639iGvMyv0De,toys,1450.0,68.0,3.0,48.0
4,1lycYGcsic2F,toys,300.0,17.0,4.0,12.0


In [26]:
products.info() # check for data types and null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127595 entries, 0 to 127594
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   product_id             127595 non-null  object 
 1   product_category_name  127119 non-null  object 
 2   product_weight_g       127570 non-null  float64
 3   product_length_cm      127570 non-null  float64
 4   product_height_cm      127570 non-null  float64
 5   product_width_cm       127570 non-null  float64
dtypes: float64(4), object(2)
memory usage: 5.8+ MB


In [27]:
products.isnull().sum() # check for missing values

product_id                 0
product_category_name    476
product_weight_g          25
product_length_cm         25
product_height_cm         25
product_width_cm          25
dtype: int64

In [29]:
# remove null values in product_name
products.dropna(subset=['product_category_name'], inplace=True)
products.isnull().sum() # check for missing values

product_id                0
product_category_name     0
product_weight_g         25
product_length_cm        25
product_height_cm        25
product_width_cm         25
dtype: int64

In [ ]:
# check for duplicates
duplicates = products.duplicated().sum()
duplicates

94338

In [31]:
# remove duplicates
products.drop_duplicates(inplace=True)
duplicates = products.duplicated().sum()
duplicates

0

In [ ]:
# save the cleaned products data
products.to_csv('data/products.csv', index=False)

### Payments Data

In [ ]:
# import payments data
payments = pd.read_csv('data/payments.csv')
payments.head(5)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,Axfy13Hk4PIk,1,credit_card,1,259.14
1,v6px92oS8cLG,1,credit_card,8,382.39
2,Ulpf9skrhjfm,1,credit_card,4,249.25
3,bwJVWupf2keN,1,credit_card,2,27.79
4,Dd0QnrMk9Cj5,1,credit_card,1,76.15


In [33]:
payments.info() # check for data types and null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127595 entries, 0 to 127594
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              127595 non-null  object 
 1   payment_sequential    127595 non-null  int64  
 2   payment_type          127595 non-null  object 
 3   payment_installments  127595 non-null  int64  
 4   payment_value         127595 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.9+ MB


In [36]:
# check for duplicates
duplicates = payments.duplicated().sum()
duplicates

0

**Overall, we merge the train and test file and then clean it by replace or removing null values and removing duplicates. Finally, we get 5 csv files that we can use in sql to drive insights.**